In [ ]:
unzip "/content/drive/MyDrive/Train_Test_Valid.zip"

Archive:  /content/drive/MyDrive/Train_Test_Valid.zip
   creating: Train_Test_Valid/test/
   creating: Train_Test_Valid/test/barrel_jellyfish/
  inflating: Train_Test_Valid/test/barrel_jellyfish/08.jpg  
  inflating: Train_Test_Valid/test/barrel_jellyfish/16.jpg  
  inflating: Train_Test_Valid/test/barrel_jellyfish/23.jpg  
  inflating: Train_Test_Valid/test/barrel_jellyfish/43.jpg  
  inflating: Train_Test_Valid/test/barrel_jellyfish/50.jpg  
   creating: Train_Test_Valid/test/blue_jellyfish/
  inflating: Train_Test_Valid/test/blue_jellyfish/01.jpeg  
  inflating: Train_Test_Valid/test/blue_jellyfish/10.jpg  
  inflating: Train_Test_Valid/test/blue_jellyfish/14.jpg  
  inflating: Train_Test_Valid/test/blue_jellyfish/19.jpg  
  inflating: Train_Test_Valid/test/blue_jellyfish/24.jpg  
  inflating: Train_Test_Valid/test/blue_jellyfish/33.jpeg  
  inflating: Train_Test_Valid/test/blue_jellyfish/47.jpg  
   creating: Train_Test_Valid/test/compass_jellyfish/
  inflating: Train_Test_Valid/te

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing libraries

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax
from keras.api._v2.keras import activations

# Data Augmenation

In [29]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [30]:
train_data = train_datagen.flow_from_directory(
'/content/Train_Test_Valid/Train',
target_size=(224,224),
batch_size=15,
class_mode='categorical')

Found 900 images belonging to 6 classes.


In [31]:
valid_data = train_datagen.flow_from_directory(
'//content/Train_Test_Valid/test',
target_size=(224, 224),
batch_size=15,
class_mode='categorical')

Found 40 images belonging to 6 classes.


# Importing VGG16

In [32]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten

In [33]:
Image_size=[224,224]

In [34]:
sol=VGG16(input_shape=Image_size + [3] , weights='imagenet' , include_top = False)

In [35]:
for i in sol.layers:
  i.trainable = False

In [36]:
y=Flatten()(sol.output)

# Constructing Fully Connected Layers

In [37]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax

In [38]:
from keras.api._v2.keras import activations
final = Dense(6, activation = 'softmax')(y)

In [39]:
from tensorflow.keras.models import Model

In [40]:
vgg16_model = Model(sol.inputs, final)

In [41]:
vgg16_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

# Compiling the Model

In [44]:
vgg16_model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['Accuracy'])

# Training the Model

In [45]:
vgg16_model.fit(train_data, epochs = 15,validation_data=valid_data)

Epoch 1/15
60/60 [==============================] - 579s 10s/step - loss: 1.1872 - Accuracy: 0.5944 - val_loss: 0.8432 - val_Accuracy: 0.6750
Epoch 2/15
60/60 [==============================] - 576s 10s/step - loss: 0.4247 - Accuracy: 0.8556 - val_loss: 0.5065 - val_Accuracy: 0.9250
Epoch 3/15
60/60 [==============================] - 591s 10s/step - loss: 0.2584 - Accuracy: 0.9222 - val_loss: 0.5073 - val_Accuracy: 0.8250
Epoch 4/15
60/60 [==============================] - 595s 10s/step - loss: 0.1483 - Accuracy: 0.9700 - val_loss: 0.4366 - val_Accuracy: 0.9000
Epoch 5/15
60/60 [==============================] - 597s 10s/step - loss: 0.0949 - Accuracy: 0.9789 - val_loss: 0.6454 - val_Accuracy: 0.8500
Epoch 6/15
60/60 [==============================] - 598s 10s/step - loss: 0.1012 - Accuracy: 0.9744 - val_loss: 0.5265 - val_Accuracy: 0.8750
Epoch 7/15
60/60 [==============================] - 599s 10s/step - loss: 0.0511 - Accuracy: 0.9933 - val_loss: 0.5299 - val_Accuracy: 0.8750
Epoch 

# save file .h5

In [ ]:
#vgg16_model.save('cnn .h5')

# Testing the Data

In [46]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [59]:
labels=['Moon_jellyfish','barrel_jellyfish','blue_jellyfish','compass_jellyfish','lions_mane_jellyfish','mauve_stinger_jellyfish']

In [63]:
img_path ='/content/Train_Test_Valid/test/compass_jellyfish/04.jpg'

In [64]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16_model.predict(np.array([x]))
preds

1/1 [==============================] - 1s 999ms/step


array([[0., 0., 0., 1., 0., 0.]], dtype=float32)

In [65]:
labels[np.argmax(preds)]

'compass_jellyfish'